## Readout sequences

In [1]:
from arbok_driver import ArbokDriver, Sample, Sequence

2024-04-09 11:46:39,047 - qm - INFO     - Starting session: e8b818c5-11e1-426d-bf20-3bc92f30a062


In [2]:
from example_configs.dummy_sample import dummy_sample
from example_sequences.dummy_readout import DummyReadout

In [3]:
qm_driver = ArbokDriver('qm_driver', dummy_sample)
dummy_sequence = Sequence('dummy_squence', dummy_sample)
qm_driver.add_sequence(dummy_sequence)

In [4]:
readout_sequence_config = {
    'parameters': {
        't_between_measurements': {
            'value': 50,
            'unit': 'cycles',
        }
    },
    'signals':{
        'qubit1':{
            'elements': {
                'sensor1': 'readout_element',
            },
            'readout_points': {
                'ref': {
                    'method': 'average',
                    'desc':'reference point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                },
                'read': {
                    'method': 'average',
                    'desc': 'redout point',
                    'observables': ['I', 'Q', 'IQ'],
                    'save_values': True
                }
            }
        },
    },
    'readout_groups': {
        'difference': {
            'qubit1__diff': {
                'method': 'difference',
                'name': 'diff',
                'args': {
                    'signal': 'qubit1',
                    'minuend': 'qubit1.ref.sensor1_IQ',
                    'subtrahend': 'qubit1.read.sensor1_IQ',
                },
            },
        }
    },
}

In [5]:
dummy_readout = DummyReadout('dummy_readout', dummy_sample, readout_sequence_config)
dummy_sequence.add_subsequence(dummy_readout)

In [6]:
qua_program = qm_driver.get_qua_program()
qm_driver.print_qua_program_to_file(
    'readout_qua_program.py', qua_program)
import compiled_readout_qua_program
compiled_readout_qua_program??

Type:        module
String form: <module 'compiled_readout_qua_program' from 'C:\\Users\\z5440260\\GitRepos\\arbok_driver\\docs\\compiled_readout_qua_program.py'>
File:        c:\users\z5440260\gitrepos\arbok_driver\docs\compiled_readout_qua_program.py
Source:     

# Single QUA script generated at 2024-04-08 12:02:21.036907
# QUA library version: 1.1.6

from qm.qua import *

with program() as prog:
    v1 = declare(int, )
    v2 = declare(fixed, )
    v3 = declare(fixed, )
    v4 = declare(fixed, )
    v5 = declare(fixed, )
    v6 = declare(fixed, )
    v7 = declare(fixed, )
    v8 = declare(fixed, )
    v9 = declare(int, )
    with infinite_loop_():
        pause()
        with for_(v1,0,(v1<1.125),(v1+0.25)):
            align()
            play("ramp"*amp(v1), "gate_1")
            wait(100, "gate_1")
            play("ramp"*amp((0-v1)), "gate_1")
            align()
            measure("measure", "readout_element", None, integration.full("x", v2, ""), integration.full("y", v3, "")